# Datathon 2024

Reto de Hey Banco

## Evolución de tópicos con respecto al tiempo


In [ ]:
!pip install bertopic

In [ ]:
!pip install spacy==3.2.0
!pip install es-core-news-sm
!pip install en_core_web_sm

  Using cached spacy-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (659 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.3
    Uninstalling thinc-8.2.3:
      Successfully uninstalled thinc-8.2.3
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.4
    Uninstalling spacy-3.7.4:
      Successfully uninstalled spacy-3.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.2.0 which is incompatible.
es-core-news-sm 3.1.0 requires spacy<3.2.0,>=3.1.0, but you have spacy 3.2.0 which is incompatible.
  Using cached spacy-3.1.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Attempting uninstall: spacy
    Found exis

## Conexión con Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Datathon2024/

/content/drive/MyDrive/Datathon2024


In [ ]:
ls

'Datathon 2024 - Reto Hey - Dataset Puìblico - Sheet1.csv'   HeyBanco.ipynb    tweets_limpios.csv
 gracias.csv                                                 my_functions.py


# Librerías

In [ ]:
import re
import pandas as pd
from bertopic import BERTopic
from umap import UMAP

from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
import numpy as np

In [ ]:
datos = pd.read_csv('Datathon 2024 - Reto Hey - Dataset Puìblico - Sheet1.csv')
teewts_limpios = pd.read_csv('tweets_limpios.csv')

In [ ]:
datos.dtypes

date     object
time     object
tweet    object
dtype: object

In [ ]:
# Prepare data
#datos.tweet = datos.apply(lambda row: re.sub(r"http\S+", "", row.tweet).lower(), 1)
#datos.tweet = datos.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.tweet.split())), 1)
#datos.tweet = datos.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.tweet).split()), 1)
timestamps = datos.date.to_list()
#tweets = datos.tweet.to_list()


In [ ]:
datos['tweet'][13]

'Si ya la bloquee y desbloquee y pude hacer el deposito en oxxo, gracias'

In [ ]:
docs = teewts_limpios['tweet_limpios']
X_train = teewts_limpios['tweet_limpios']

In [ ]:
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = sentence_model.encode(X_train, show_progress_bar=True)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
# Train BERTopic
topic_model = BERTopic().fit(X_train, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(X_train, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=150, n_components=5, min_dist=0.0, metric='cosine').fit_transform(embeddings)

hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)
topic_model = BERTopic(hdbscan_model=hdbscan_model, n_gram_range=(1,2))
topics, probs = topic_model.fit_transform(docs)


Quitar outliers

In [ ]:
new_topics = topic_model.reduce_outliers(docs, topics)
topic_model.update_topics(docs, topics=new_topics)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

2024-05-04 21:32:01,428 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
topics_over_time = topic_model.topics_over_time(X_train, timestamps)
topic_model.visualize_topics_over_time(topics_over_time)

2024-05-04 21:28:53,169 - BERTopic - WARNING: There are more than 100 unique timestamps (i.e., 324) which significantly slows down the application. Consider setting `nr_bins` to a value lower than 100 to speed up calculation. 


Se prueba diferentes configuraciones con el fin de obtener mejores tópicos.
Voy a eliminar la palabra gracias pues se repite mucho en los textos.

In [ ]:
pip install emoji==2.7.0

In [ ]:
def remove_emojis(data):
  emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
  return re.sub(emoj, '', data)

In [ ]:
datos_sg = pd.read_csv('gracias.csv')

In [ ]:
datos_sg.dropna(subset=['tweet_gracias'],inplace=True)


In [ ]:
datos_sg.head(12)

,date,time,tweet,tweet_limpios,bigram_text,topic,tweet_gracias
0,2023-01-01,0,"Resuelto, muchísimas gracias , excelente servi...",Resuelto muchísimas gracias excelente servici...,Resuelto_muchísimas muchísimas_gracias gracias...,0,Resuelto muchísimas excelente servicio siempre
1,2023-01-02,0,"Muchas gracias, espero su dm",Muchas gracias espero dm,Muchas_gracias gracias_espero espero_dm,1,Muchas espero dm
2,2023-01-02,0,Muchas gracias!,Muchas gracias!,Muchas_gracias!,-1,Muchas
3,2023-01-02,0,Algo similar me paso. Quería renovar mi token ...,Algo similar paso Quería renovar token poder ...,Algo_similar similar_paso paso_Quería Quería_r...,-1,Algo similar paso Quería renovar token poder ...
4,2023-01-02,0,Yeeeei! a través de mi cuenta en acabo de cont...,Yeeeei! través cuenta acabo contratar cuenta ...,Yeeeei!_través través_cuenta cuenta_acabo acab...,-1,Yeeeei! través cuenta acabo contratar cuenta ...
5,2023-01-02,0,Esta es la señal que necesitan para tener el m...,Esta señal necesitan tener mínimo dinero banamex,Esta_señal señal_necesitan necesitan_tener ten...,-1,Esta señal necesitan tener mínimo dinero banamex
6,2023-01-03,0,Gracias 😁,Gracias 😁,Gracias_😁,1,😁
7,2023-01-03,0,Muchas gracias 😊,Muchas gracias 😊,Muchas_gracias gracias_😊,1,Muchas 😊
8,2023-01-03,0,"Yo no sé mucho, pero te recomiendo la Inversió...",Yo sé mucho recomiendo Inversión De mejores r...,Yo_sé sé_mucho mucho_recomiendo recomiendo_Inv...,0,Yo sé mucho recomiendo Inversión De mejores r...
9,2023-01-04,0,Gracias por la playera que me dieron en Galerí...,Gracias playera dieron Galerías GDL,Gracias_playera playera_dieron dieron_Galerías...,1,playera dieron Galerías GDL


In [ ]:
X_train_g = datos_sg['tweet_gracias']

In [ ]:
datos_sg.reset_index(drop=True, inplace=True)

In [ ]:
X_train_g[1]

' Muchas espero dm'

In [ ]:
docs=datos_sg['tweet_gracias']
X_train_g = docs

In [ ]:
sentence_model_g = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings_g = sentence_model_g.encode(datos_sg['tweet_gracias'], show_progress_bar=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
# Train BERTopic
topic_model_g = BERTopic().fit(X_train_g, embeddings_g)

# Run the visualization with the original embeddings
topic_model_g.visualize_documents(X_train_g, embeddings=embeddings_g)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
#reduced_embeddings_g = UMAP(n_neighbors=200, n_components=8, min_dist=0.0, metric='cosine').fit_transform(embeddings_g)

hdbscan_model_g = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)
topic_model_g = BERTopic(hdbscan_model=hdbscan_model_g, n_gram_range=(1,2))
topics_g, probs = topic_model_g.fit_transform(docs)

In [ ]:
len(topics_g)

793

In [ ]:
#new_topics = topic_model_g.reduce_outliers(docs.values, topics_g)
#topic_model_g.update_topics(docs, topics=new_topics)
topic_model_g.visualize_documents(docs)#, reduced_embeddings=reduced_embeddings_g)

In [ ]:
from my_functions import *
preprocesador = preprocesaTexto(idioma='es', _tokeniza=False, _muestraCambios=False, _quitarAcentos=True,
                                _remueveStop=True, _stemming=False, _lematiza=True, _removerPuntuacion=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
txt_prep = []
for i in range(len(datos['tweet'])):
  txt_prep.append(preprocesador.preprocesa(datos['tweet'][i]))

In [ ]:
docs = txt_prep
X_train_g = txt_prep

In [ ]:
sentence_model_g = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings_g = sentence_model_g.encode(txt_prep, show_progress_bar=True)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
# Train BERTopic
topic_model_g = BERTopic().fit(X_train_g, embeddings_g)

# Run the visualization with the original embeddings
topic_model_g.visualize_documents(X_train_g, embeddings=embeddings_g)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings_g = UMAP(n_neighbors=70, n_components=5, min_dist=0.0, metric='cosine').fit_transform(embeddings_g)

hdbscan_model_g = HDBSCAN(min_cluster_size=8, metric='euclidean', prediction_data=True)
topic_model_g = BERTopic(hdbscan_model=hdbscan_model_g, n_gram_range=(1,2))
topics_g, probs = topic_model_g.fit_transform(docs)

In [ ]:
#new_topics = topic_model_g.reduce_outliers(docs, topics_g)
#topic_model_g.update_topics(docs, topics=new_topics)
topic_model_g.visualize_documents(docs, reduced_embeddings=reduced_embeddings_g)

In [ ]:
topics_over_time = topic_model_g.topics_over_time(X_train_g, timestamps)
topic_model_g.visualize_topics_over_time(topics_over_time)

2024-05-05 06:54:17,633 - BERTopic - WARNING: There are more than 100 unique timestamps (i.e., 324) which significantly slows down the application. Consider setting `nr_bins` to a value lower than 100 to speed up calculation. 


In [ ]:
datos[datos.date=='2023-09-01']

,date,time,tweet
514,2023-09-01,18:06:14,Muy felices y contentos por nuestros clientes ...
515,2023-09-01,16:25:34,"Perfecto, ya pude entrar a la App y como según..."
516,2023-09-01,00:51:39,"Yo la tengo, esta bien padre."


In [ ]:
datos.iloc[531:540,[0,2]]

,date,tweet
531,2023-09-15,A mi me sirvió actualizar mi versión de Androi...
532,2023-09-15,Gracias justo eso me funciono
533,2023-09-16,Yo elimine el cache en los ajustes de la app y...
534,2023-09-16,Si me funcionó!!! Muchas gracias
535,2023-09-16,"Entren a ajustes de la app , borren el cache y..."
536,2023-09-16,Esto me funcionó a mi
537,2023-09-16,Muchas gracias Hey! Ya quedó resuelto. 👌👌👌
538,2023-09-16,Gracias ! Lo intentare 😁
539,2023-09-17,Gracias! También me funcionó reiniciando 🙌


In [ ]:
datos[datos.date=='2023-03-13']

,date,time,tweet
215,2023-03-13,NaN,Gracias!
216,2023-03-13,NaN,Gracias :)
217,2023-03-13,NaN,Me siguen hablando en horario laboral.El que s...
218,2023-03-13,NaN,Exacto puntos lealtad que solo se cambian en s...
219,2023-03-13,NaN,Verdaderamente el mejor banco que hay 💖✨ los t...
220,2023-03-13,NaN,"Bueno, es que yo si pude ya."
221,2023-03-13,NaN,Excelente!!
222,2023-03-13,NaN,Excelente
223,2023-03-13,NaN,Muy bien excelente
224,2023-03-13,NaN,"Me gane un mes de Spotify y no soy usuario 😄, ..."


In [ ]:
datos.iloc[1:200,[0,2]]

,date,tweet
1,2023-01-02,"Muchas gracias, espero su dm"
2,2023-01-02,Muchas gracias!
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...
5,2023-01-02,Esta es la señal que necesitan para tener el m...
...,...,...
195,2023-03-06,"Gracias, ya lo resolví jajaja, me equivoqué"
196,2023-03-06,Quedó aclarado el tema. Gracias.
197,2023-03-07,Bank run!!!
198,2023-03-07,Les respondí por privado. Saludos!


# Otras configuraaciones

In [ ]:
# Train BERTopic
topic_model_g = BERTopic(top_n_words=15).fit(X_train_g, embeddings_g)

# Run the visualization with the original embeddings
topic_model_g.visualize_documents(X_train_g, embeddings=embeddings_g)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings_g = UMAP(n_neighbors=75, n_components=5, min_dist=0.0, metric='cosine').fit_transform(embeddings_g)

hdbscan_model_g = HDBSCAN(min_cluster_size=8, metric='euclidean', prediction_data=True)
topic_model_g = BERTopic(hdbscan_model=hdbscan_model_g, n_gram_range=(1,4))
topics_g, probs = topic_model_g.fit_transform(docs)

In [ ]:
new_topics = topic_model_g.reduce_outliers(docs, topics_g)
topic_model_g.update_topics(docs, topics=new_topics)
topic_model_g.visualize_documents(docs, reduced_embeddings=reduced_embeddings_g)

2024-05-05 06:13:36,436 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
topics_over_time = topic_model_g.topics_over_time(X_train_g, timestamps)
topic_model_g.visualize_topics_over_time(topics_over_time)

2024-05-05 06:13:36,730 - BERTopic - WARNING: There are more than 100 unique timestamps (i.e., 324) which significantly slows down the application. Consider setting `nr_bins` to a value lower than 100 to speed up calculation. 


# *Tendencias del mercado:*

Identificar temas populares o emergentes en los tweets puede ayudar a anticipar tendencias del mercado y ajustar la estrategia empresarial en consecuencia.
